In [10]:
import tensorflow as tf
import numpy as np
import sys
import os
import functools
from my_load_data import load_dataset_fn
#OPTIONAL FOR DEBUGGING
#import import_ipynb

In [11]:
#Parameters of photo
total_num_batches = 124
img_size = 256
num_channels = 3
#Hyper parameters for network
num_classes = 2
batch_size = 10
total_steps = 2
learning_rate = 1e-4
"""Config variable named params, used for forwarding parameters through
whole model"""
#Params used as config file
params = {"learning_rate": learning_rate,
          "img_size": img_size,
          "num_channels":num_channels,
          "num_classes":num_classes,
          "batch_size":batch_size,
          "total_batches":total_num_batches,
          "total_steps":total_steps}

#### Following code serves for nice and smart defining model as reusable code

In [12]:
def doublewrap(function):
    """
    A decorator of decorator, allowing use of lazy property if no arguments are provided
    """
    @functools.wraps(function)
    def decorator(*args, **kwargs):
        if len(args) == 1 and len(kwargs) == 0 and callable(args[0]):
            return function(args[0])
        else:
            return lambda wrapee: function(wrapee, *args, **kwargs)
    return decorator

@doublewrap
def define_scope(function, scope = None, *args, **kwargs):
    """
    Lazy decorator, optimizes code by loading class Model parts only once to memory
    Also its groups tf.Graph, in tensorboard into smaller, more readable parts
    """    
    attribute = '_cache_' + function.__name__
    name = scope or function.__name__
    
    @property
    @functools.wraps(function)
    def decorator(self):
        if not hasattr(self, attribute):
            #Sorting Graph by Var_scope
            with tf.variable_scope(name, *args, **kwargs):
                setattr(self, attribute, function(self))
                print("Initialized Model.{}".format(name))
        return getattr(self, attribute)
    
    return decorator

def accuracy(predictions, labels):
    #Not working, figure out how to make it work for not hot max code
    return (100.0 * np.sum(tf.argmax(predictions, 1) == tf.argmax(labels, 1))
        / predictions.shape[0])

#def feed_dict(step, train=True):
 #   """Used to get input data for network, make it so it can take train or test as input
 #   and return proper dataset and label"""
 #   if train == True:
 #       return "error"    

def save_restore():
    """
    To do
    """
    #ckpt = tf.train.get_checkpoint_state(os.path.dirname('checkpoints/checkpoint'))
        #if ckpt and ckpt.model_checkpoint_path:
             #saver.restore(sess, ckpt.model_checkpoint_path)
    #saver.restore(sess, 'checkpoints/skip-gram-10000')
    pass

##### Defining model as reusable function

In [54]:
class Model:
    """
    Model of neural network with all functionalities
    """
    def __init__(self, image, labels_class, labels_loc, params, mode=None):
        #Variables
        self.data = image
        self.target_class = labels_class
        self.target_loc = labels_loc
        self.img_size = params["img_size"]
        self.num_channels = params["num_channels"]
        self.num_classes = params["num_classes"]
        self.mode = mode
        self.global_step = tf.Variable(0, dtype=tf.int32,
                                       trainable=False, name='global_step')
        #Functions
        self.prediction
        self.classifier
        self.optimize_cl
        self.loss_cl
        self.localizer
        self.optimize_loc
        self.loss_loc
        #self.summary
        
    @define_scope
    def prediction(self):
        """
        Main body of neural network, takes data and labels as input,
        returns feature map of photo
        """
        #INPUT LAYER
        input_layer = tf.reshape(self.data,[-1, self.img_size, self.img_size, self.num_channels])
        #1 conv layer
        conv1 = tf.layers.conv2d(inputs = input_layer, 
                                 filters = 16,
                                 kernel_size = 5,
                                 strides = 1,
                                 padding = "same",
                                 activation = tf.nn.relu)

        #1 pool layer, img size reduced by 1/4
        pool1 = tf.layers.max_pooling2d(inputs=conv1,
                                        pool_size = 2, 
                                        strides = 2,
                                        padding = "same")

        #Resizing of pool1 output
        pool_1_flat = tf.reshape(pool1,[-1,int(self.img_size * self.img_size * 16 / 4)])

        #First Dense layer
        dense = tf.layers.dense(inputs = pool_1_flat,
                                units = 128,
                                activation = tf.nn.relu,
                                name = "Dense1")
        #Second Dense layer
        dense2 = tf.layers.dense(inputs = dense,
                                 units = 64,
                                 activation = tf.nn.relu,
                                 name = "Dense2")
        
        return dense2
    
    @define_scope
    def classifier(self):
        """
        Assigns class to result
        """
        #Conversion to num_classes output size
        logits = tf.layers.dense(inputs = self.prediction,
                                 units = self.num_classes,
                                 activation = tf.nn.relu)
        # Softmax output of the neural network
        y_pred = tf.nn.softmax(logits=logits)
        #Optional hotmax output, obsolete, used in classifier
        #y_pred_cls = tf.argmax(y_pred, axis=1)
        return y_pred
    
    @define_scope
    def optimize_cl(self):
        """
        Optimizer of network, call after model.classifier to optimize classifier network
        """
        cross_entropy_cl =  tf.nn.softmax_cross_entropy_with_logits_v2(labels=self.target_class, logits=self.classifier)
        optimizer_cl = tf.train.AdamOptimizer(learning_rate = params["learning_rate"])
        return optimizer_cl.minimize(cross_entropy_cl)
    
    @define_scope
    def loss_cl(self):
        """
        Function returning loss for classifier
        """
        loss_cl = tf.losses.softmax_cross_entropy(self.target_class, self.classifier)
        return loss_cl
    
    @define_scope
    def localizer(self):
        """
        Function tries to obtain localization of marker on photo
        """
        #Conversion to num_classes output size
        logits_loc = tf.layers.dense(inputs = self.prediction,
                                 units = 4,
                                 activation = tf.nn.relu)
        
        # Softmax output of the neural network
        loc = tf.nn.softmax(logits=logits_loc)
        return loc
    
    @define_scope
    def optimize_loc(self):
        """
        Optimizes localizer
        """
        cross_entropy_loc =  tf.nn.softmax_cross_entropy_with_logits_v2(labels=self.target_loc, logits=self.localizer)
        optimizer_loc = tf.train.AdamOptimizer(learning_rate = params["learning_rate"])
        return optimizer_loc.minimize(cross_entropy_loc)
    
    @define_scope
    def loss_loc(self):
        """
        Returns loss for localizator
        """
        loss_loc = tf.losses.mean_squared_error(self.target_loc, self.localizer)
        tf.summary.scalar = ('loss_loc', loss_loc)
        return loss_loc
    """    
    @define_scope
    def summary(self):
        with tf.name_scope('summary'):
            tf.summary.scalar("loss",loss_cl)
            tf.summary.histogram("histogram loss",loss_cl)
            #Merge all summaries
    summary = tf.summary.merge_all()
        pass 
            
    """

In [63]:
def main(params, mode = None):
    total_steps = params["total_steps"]
    graph = tf.Graph()
    #merged =tf.summary.merge_all()
    with graph.as_default():

        print("Starting Session")
        with tf.Session(graph=graph) as sess:
            
            #Creating summary writer 
            writer = tf.summary.FileWriter("./results")
            
            #Placeholders for dataset and labels
            with tf.name_scope('Image'):
                ph_image = tf.placeholder(tf.float32, [None, params["img_size"], params["img_size"], params["num_channels"]])
            with tf.name_scope('Label_class'):
                ph_labels_class = tf.placeholder(tf.float32, [None, params["num_classes"]])
            with tf.name_scope('Label_localization'):
                ph_labels_loc = tf.placeholder(tf.float32, [None, 4])

            #Initialization of Model
            model = Model(ph_image, ph_labels_class, ph_labels_loc,
                          params, mode = None)

            #Initialization of variables
            sess.run(tf.global_variables_initializer())
            print('Variables initialized')

            #Creating save for model session for future saving and restoring model
            saver = tf.train.Saver()
            save_path=saver.save(sess,"my_net/save_net.ckpt")
            print("Session saved to path:",save_path)
            
            #Training
            print("Starting Training")
            for step in range(1,total_steps+1):
                dataset, labels = load_dataset_fn(np.round(step/total_steps*params["total_batches"])+1)
                #
                feed_dict = {ph_image: dataset, 
                             ph_labels_class: labels[:, [0,1]], 
                             ph_labels_loc: labels[:,[2,3,4,5]]}
                
                del dataset, labels
                
                #########SESS.RUN HERE###############
                loss,  _ = sess.run([model.loss_cl, model.optimize_cl], 
                                            feed_dict= feed_dict)
                
                #Saving usefull info
                #writer.add_summary(loss, step)

                #Printing data every 10th step
                if (step % 1 == 0):
                    print('Loss at step %d: %f' % (step, loss))
                    # print("Accuracy: {0:2.2f}".format(accuracy(pred, labels)))
                if (step + 1) % 1000==0:
                    saver.save(sess, 'checkpoint_directory/model_name',
                               global_step=model.global_step)
        
        
        print("Finnished session")
        writer.add_graph(graph)
        writer.close()
        print("Closed summary, work finnished")
        



In [64]:
tf.reset_default_graph()
if __name__ == '__main__':
     main(params)

Starting Session
Initialized Model.prediction
Initialized Model.classifier
Initialized Model.optimize_cl
Initialized Model.loss_cl
Initialized Model.localizer
Initialized Model.optimize_loc
Initialized Model.loss_loc
Variables initialized
Session saved to path: my_net/save_net.ckpt
Starting Training
Loss at step 1: 0.688472
Loss at step 2: 0.750382
Finnished session
Closed summary, work finnished


In [ ]:
###Paste into conda prompt
#   tensorboard --logdir="results/"